# Cube Shuffling Error

The goal: make a program that takes in an array of 3D cubes and spits out another array of 3D cubes. The first array was organized by wavelength and had (incidence, emission, azimuth) as the points. The new array should be organized by azimuth and have (incidence, emission, wavelength) as the points. Simple, right? 

Well, the cubes are certainly shuffling and organazing their data, I can even access it in the cube objects themselves. However, after the program's done running, some of the created *files* do not have any values in them. But only some of them. Degrees 0 10 20 30 40 45 50 55 65 and 180 have their values, seemingly correctly. 

The code is below.

```
#include "../../Jcrap.h"
#include <cmath>


double tolerance(3.); //in degrees, allowing for things barely outside of pixels or between two
double resolution(5.);

int main(int argn, char **argv) //standard "main takes input from command line."
    //Command line usage should look like "crimsoneye [Cubes]".
    {
    vector<cube> fullsignalcubes(argn-1); //Create a cube vector to hold all input cubes.
    cout << "argn equals " << argn << "\n";
    for (int i(1);i<argn;i++)
        fullsignalcubes.at(i-1)=cube(argv[i]); //Create a cube from the argument's .Jcube file, feed it into
    //the vector.

    string wavsR[8] = {"0.93","1.08","1.27","1.59","2.01","2.69","2.79","5.00"};

    cube iew_template(100./resolution+1, 90./resolution+1, 8);
    for (int j(0);j<iew_template.N(X);j++)
        iew_template.Axis(X,j)=0.+double(j)*resolution;  // incidence angle "i"
    for (int j(0);j<iew_template.N(Y);j++)
        iew_template.Axis(Y,j)=0.+double(j)*resolution;  // emission angle "e"
    for (int j(0);j<iew_template.N(Z);j++)
        iew_template.Axis(Z,j)=stod(wavsR[j]);  // wavelengths, the primary thing to swap.

    vector<cube> swap_wave_azimuth(180./resolution+1, iew_template); // one for each azimuth.

    for (int n(0);n<fullsignalcubes.size();n++) { //For every input cube...
        for (int x(0);x<fullsignalcubes.at(0).N(X);x++) {
            for (int y(0);y<fullsignalcubes.at(0).N(Y);y++) { //For every pixel in the cubes...
                for (int z(0);z<fullsignalcubes.at(0).N(Z);z++) {
                    swap_wave_azimuth.at(z)(x,y,n) = fullsignalcubes.at(n)(x,y,z); //The actual swap
                }
            }
        }
    }

    for (int w(0);w<swap_wave_azimuth.size();w++) { //Go over every cube to prepare for printing.
        string outname("iephi_consolidate_byWavelength_");
        outname += to_string(w*5); //Title needs the degree in it
        outname += string("deg.Jcube");
        cout << to_string(swap_wave_azimuth.at(w)(5,5,1)) << " "; //Bizarrely we can still access values HERE.
        //attempted: create a new cube fill and fill it with values. New cube's values still read
        //properly. Writing from new cube still results in problems.
        //I can access the value after writing so it isn't being destroyed.
        swap_wave_azimuth.at(w).write(outname);
    }
    cout << to_string(swap_wave_azimuth.at(36)(5,5,1)) << " "; //Bizarrely we can still access values
    swap_wave_azimuth.at(36).write("Test.Jcube"); //Create a file for every azimuth degree.
}

```

For clarity's sake this is as follows:

1) Read data from provided cubes. We know this is functional as the data we do get out matches the data we put in, as well as using this exact same setup for other codes. 

2) Set up the destination cubes, which will be shaped differently from the input cubes as they have a different z axis. We know this is done properly since we don't get any errors thrown, and also because even when the output is filled with NANs the axes still have their values in them: (incidence, emission, wavelength) can be found with Jcubeviwer even on an empty cube. 

3) Set up a vector of the destination cubes. We know this is working since cube 180deg always generates properly, and it is the last one.

4) For every input point, map it to the destination, just swap the "coordinates". In this case this is manifested by swapping the z-coordinate and the n-"Coordinate". 

5) Create a title for the output file and write it. In this area there are some printing functions that are there to find some errors: they don't find any, both before and after the writing occur the values within any of the cubes can be accessed.

So, as best as I can tell, steps 1-4 go through flawlessly, because afterward I can pick individual values out of the `swap_wave_azimuth` vector of cubes at will and get a result. (There are *occasional* NANs but those were because the original data had some. Point 5,5,1, which I am testing above, never has NANs). So the data exists and since the files that *do* work have the right data, presumably it's in the right arrangement as well. 

I ran into a bit of a block when I couldn't figure out how to edit the actual writing code to prove it for problems: `Jio.c++` for instnace. I tracked down the actual writing step to `(*this)(x,y,z)`. But since the cube itself is the one calling the function to write (and the subfunctions it takes to get there)  it should easily be accessing its points. And while the function its using to write is paralellized, it's only paralellized when it's called, every individual cube we write should be able to act without being interfered. (And it's also deterministic, which angles are able to be saved as .Jcube files and which are not). So I am quite stumped. 

It is at least partially working, as the axes values for the wavelength are put in every time. It's like, for some reason, the write function is writing as though the values had never been put in, despite me being able to access them in the middle of the program with ease. What's up with that?

FOUND IT! 

Fix: check when input values are NaN (isnan()). The thing is, this is only a problem when the FIRST data value in the chain is NaN, which seems to ruin eveyrthing, NANs in the middle aren't an issue. however it is easiest to check for all NaNs, the fix was setting the input values to -1 if they were NaN. 

As for WHY this happens... probably a quirk of the write() function, if I had to guesS? 